### Predictive Analytics Exam Bonus Task [PGR304] ###
2023 Autumn

Canadidate 2002

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
from pandas import Series,DataFrame
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

### Data pre-processing ###

In [9]:
# Load the dataset
data = pd.read_csv('DataSet_4_Exam.csv')
# Rename the unknown column
data.rename(columns={'Unnamed: 0': 'Experiment Epoch'}, inplace=True)
data.head()

,Experiment Epoch,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,...,Feature16,Feature17,Feature18,Feature19,Feature20,Feature21,Feature22,Feature23,Feature24,Result
0,0,0.134842,1191.742676,41.507889,4.691728,3.049478,2.138430,4.305591,4272.666992,139.076599,...,0.510055,0.806333,0.449914,841.148804,23.524372,17.455412,27.178885,1.0,5,2
1,1,0.118853,1206.649292,41.781891,5.887320,3.350271,2.698908,4.039053,4274.158691,139.125168,...,1.296457,0.238310,0.425843,730.358582,20.788883,12.424165,27.599531,1.0,5,2
2,2,0.538094,2164.658936,72.728867,16.221436,12.332704,7.785572,3.701066,4268.880859,138.953369,...,5.490809,2.425885,0.665966,1543.514160,35.843491,22.637938,34.238338,5.0,5,3
3,3,0.201133,2034.619629,72.419518,6.974368,3.560071,2.219646,3.950444,4291.068359,139.675568,...,1.379292,1.112216,0.401076,674.120667,20.867197,15.904705,27.892487,1.0,5,1
4,4,0.109943,1207.436646,42.912621,9.026774,2.954104,3.698219,3.732934,4258.276367,138.654419,...,1.487997,0.887740,0.489169,651.093445,22.442629,23.063429,29.206781,3.0,7,2


In [10]:
#Replacing all the NaN values with median 
data.fillna(data.median(),inplace= True)

# Check if there is any missing null data again
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Experiment Epoch  480 non-null    int64  
 1   Feature1          480 non-null    float64
 2   Feature2          480 non-null    float64
 3   Feature3          480 non-null    float64
 4   Feature4          480 non-null    float64
 5   Feature5          480 non-null    float64
 6   Feature6          480 non-null    float64
 7   Feature7          480 non-null    float64
 8   Feature8          480 non-null    float64
 9   Feature9          480 non-null    float64
 10  Feature10         480 non-null    int64  
 11  Feature11         480 non-null    float64
 12  Feature12         480 non-null    float64
 13  Feature13         480 non-null    float64
 14  Feature14         480 non-null    float64
 15  Feature15         480 non-null    float64
 16  Feature16         480 non-null    float64
 1

In [11]:
# Separate features and target
X = data.drop(['Result', 'Experiment Epoch'], axis=1)
y = data['Result']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.03, random_state=42, stratify=y)

# Feature Scaling - Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Check for NaN values and replace them with the mean of the column
# np.isnan(X_scaled).any() returns True if any NaN values are present
if np.isnan(X_train_scaled).any():
    col_mean = np.nanmean(X_train_scaled, axis=0)
    inds = np.where(np.isnan(X_train_scaled))
    X_train_scaled[inds] = np.take(col_mean, inds[1])

# Check for inf values and replace them with a large finite number
# np.isinf(X_scaled).any() returns True if any infinite values are present
if np.isinf(X_train_scaled).any():
    X_train_scaled[X_train_scaled == np.inf] = np.finfo(np.float64).max
    X_train_scaled[X_train_scaled == -np.inf] = np.finfo(np.float64).min

# Check the shape of the training set
X_train.shape, y_train.shape

((465, 24), (465,))

#### Modeling Training ####

In [14]:
# Train a Decision Tree Classifier
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train_scaled, y_train)

# Predictions from the Decision Tree model
dt_predictions = dt_classifier.predict(X_test_scaled)
dt_classification_report = classification_report(y_test, dt_predictions)
dt_confusion_matrix = confusion_matrix(y_test, dt_predictions)

print(dt_classification_report) 
print(dt_confusion_matrix) 

              precision    recall  f1-score   support

           1       0.88      0.88      0.88         8
           2       0.83      0.83      0.83         6
           5       1.00      1.00      1.00         1

    accuracy                           0.87        15
   macro avg       0.90      0.90      0.90        15
weighted avg       0.87      0.87      0.87        15

[[7 1 0]
 [1 5 0]
 [0 0 1]]


In [15]:
# Train a K-Nearest Neighbors Classifier
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train_scaled, y_train)

# Predictions from the KNN model
knn_predictions = knn_classifier.predict(X_test_scaled)
knn_classification_report = classification_report(y_test, knn_predictions)
knn_confusion_matrix = confusion_matrix(y_test, knn_predictions)
print(knn_classification_report) 
print(knn_confusion_matrix)

              precision    recall  f1-score   support

           1       0.80      1.00      0.89         8
           2       1.00      0.67      0.80         6
           5       1.00      1.00      1.00         1

    accuracy                           0.87        15
   macro avg       0.93      0.89      0.90        15
weighted avg       0.89      0.87      0.86        15

[[8 0 0]
 [2 4 0]
 [0 0 1]]
